In [2]:
# Зависимости
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor 

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [3]:
# Инициализируем все известные генераторы случаынйх чисел / Setting all known random seeds
my_code = "Sulima"
seed_limit = 2 ** 32
my_seed = int.from_bytes(my_code.encode(), "little") % seed_limit

os.environ['PYTHONHASHSEED']=str(my_seed)

random.seed(my_seed)

np.random.seed(my_seed)

In [4]:
# Читаем данные из файла
train_data = pd.read_csv("datasets/rus_stocks_funds_train.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'datasets/rus_stocks_funds_train.csv'

In [ ]:
train_data.head()

In [ ]:
#pairplot(train_data)

In [ ]:
# Определим размер валидационной выборки
val_size = round(0.2*len(train_data))
print(val_size)

In [ ]:
# Создадим обучающую и валидационную выборки
random_state = my_seed
train, val = train_test_split(train_data, test_size=val_size, random_state=random_state)
print(len(train), len(val))

In [ ]:
# Значения в числовых столбцах преобразуем к отрезку [0,1].
# Для настройки скалировщика используем только обучающую выборку.
y_column = 'Debt/Assets'
x_columns = [e for e in train_data.columns if e != y_column]

ct = ColumnTransformer(transformers=[('numerical', MinMaxScaler(), x_columns)], remainder='passthrough')

ct.fit(train)

In [ ]:
# Преобразуем значения, тип данных приводим к DataFrame
sc_train = pd.DataFrame(ct.transform(train))
sc_val = pd.DataFrame(ct.transform(val))

In [ ]:
# Устанавливаем названия столбцов
column_names = list(x_columns) + [y_column]
sc_train.columns = column_names
sc_val.columns = column_names

In [ ]:
sc_train

In [ ]:
# Отберем необходимые параметры
x_train = sc_train[x_columns]
x_val = sc_val[x_columns]

y_train = (sc_train[y_column].values).flatten()
y_val = (sc_val[y_column].values).flatten()

In [ ]:
# Создадим простую модель логистической регрессии
model = TransformedTargetRegressor(regressor=LinearRegression(), transformer=MinMaxScaler())

In [ ]:
# Обучим модель
model.fit(x_train, y_train)

In [ ]:
# Проверим работу обученной модели на валидационной выборке
pred_val = model.predict(x_val)
mse = mean_squared_error(y_val, pred_val)
print(mse)

In [ ]:
pred_val

In [ ]:
test = pd.read_csv("datasets/rus_stocks_funds_test.csv")

In [ ]:
test[y_column] = 0.0

In [ ]:
test

In [ ]:
sc_test = pd.DataFrame(ct.transform(test))
sc_test.columns = column_names

In [ ]:
x_test = sc_test[x_columns]

In [ ]:
test[y_column] = model.predict(x_test)

In [ ]:
test.head()

In [ ]:
test.to_csv('task5_results/rus_stocks_funds_result.csv', index=False)